In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import sqlite3
import time
pd.options.display.float_format = '{:,.1f}'.format

In [2]:
# 드라이버 객체 생성
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('diable-gpu')
driver = webdriver.Chrome('chromedriver.exe', chrome_options=options)

In [3]:
# 링크 주소 가져오기
def getLink(n): 
    if n < 50:
        selector = ['tr[data-index="'+str(n)+'"]','td.img','div','a.newLink._link']
        selector = ' > '.join(selector)
        link = driver.find_element_by_css_selector(selector).get_attribute('href')
        return link
    else:
        return None
    
# 페이지 클릭하기
# 아무 페이지나 실행되진 않고 순차적으로 실행해야 함
def clickPage(page):
    selector = ['a[data-page="'+str(page)+'"]']
    selector = ' > '.join(selector)
    driver.find_element_by_css_selector(selector).click()

In [4]:
# 데이터 적재할 DB 생성
conn = sqlite3.connect('skencar.db')
cur = conn.cursor()

In [9]:
# 데이터 적재할 테이블 생성
cur.execute('drop table 링크')
cur.execute('create table 링크(번호, 주소)')
cur.execute('delete from 링크')

In [10]:
# 홈페이지 메인 화면으로 접속
url = 'http://www.encar.com/index.do'
driver.get(url)

# 크롤링 메인 화면으로 접속
driver.find_element_by_id('manufact').click()
selector = ['#manufactListText','ul:nth-of-type(2)','li','a']
selector = ' > '.join(selector)
driver.find_element_by_css_selector(selector).click()
driver.execute_script("goSearch('car')")
time.sleep(2)
Select(driver.find_element_by_id('pagerow')).select_by_value('50')

In [ ]:
# 링크 긁어오기
pages = 110
for page in range(1,pages):
    if page != 1: clickPage(page)
    time.sleep(2)
    for i in range(50):
        cur.execute('insert into 링크 values (?,?)', (i, getLink(i)))
    if page%10 == 1:
        print('--- {:3} Page... Done. ---'.format(page))

In [12]:
# 적재한 데이터 출력
cur.execute('select * from 링크')
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['번호','링크'])['링크']